<a href="https://colab.research.google.com/github/AiDarkEzio/Project-Jupyter-NoteBooks/blob/main/Stable_Diffusion_03(lora%2C_controlnet%2C_VAE)_%2B_export_to_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **StableDiffusion GoogleColab version by @marat_ai**


_You don't need additional Google Drive storage because uploaded models are not stored on your Google Drive. After the session ends, all data will be deleted. You can either download ControlNet models on your own or let me know if you want me to make it for you._

In [ ]:
#@markdown # **STEP 1**
#@markdown ## Requirements
from IPython.display import clear_output

print('[1;32mInstalling requirements...')
#%pip install --quiet blendmodes accelerate basicsr fonts font-roboto gfpgan gradio invisible-watermark numpy omegaconf opencv-contrib-python requests piexif Pillow pytorch-lightning==1.7.7 realesrgan scikit-image==0.19 timm==0.4.12 transformers==4.25.1 torch einops jsonmerge clean-fid resize-right torchdiffeq kornia lark inflection GitPython torchsde safetensors psutil rich
#%cd /content/
!git clone -q --branch main https://github.com/AlexSgt/lite_colab
%cd /content/lite_colab
%pip install -r requirements.txt --quiet
!sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/lite_colab/modules/sd_models.py
!sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/lite_colab/startfk.py
!sed -i "s@map_location='cpu'@map_location='cuda'@" /content/lite_colab/modules/extras.py
!cp /content/lite_colab/assets/blocks.py /usr/local/lib/python3.10/dist-packages/gradio/blocks.py

clear_output()
print('[1;32mDone!')
#!python launch.py --share --theme dark --api --ckpt-dir /content/models/ --disable-safe-unpickle --enable-insecure-extension-access --opt-sdp-attention --disable-console-progressbars --no-download-sd-model

In [ ]:
#@markdown # **STEP 2**
#@markdown ## Choose your model
#@markdown _Paste link from huggingface.co or leave the default model_

MODEL_LINK = "https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1.safetensors" #@param {type:"string"}
safetensors = True #@param {type:"boolean"}

if MODEL_LINK != "":
  pth = '/content/lite_colab/models/Stable-diffusion/'
  if not safetensors:
    modelname="model.ckpt"
  else:
    modelname="model.safetensors"
  dwnld = pth + modelname
  print('[1;32mDownload model...')
  !gdown --fuzzy -O $dwnld "$MODEL_LINK"
  clear_output()
  print('[1;32mDone!')
else:
  print('[1;31mPaste model link and try again!')



In [ ]:
#@markdown # **STEP 2.1** (optional)
#@markdown ## Choose Lora model





import os
import urllib.parse
import requests

Lora_model_link1 = "https://civitai.com/api/download/models/112406" #@param {type:"string"}
Lora_model_link2 = '' #@param {type:"string"}
Lora_model_link3 = '' #@param {type:"string"}
Lora_model_link4 = '' #@param {type:"string"}
Lora_model_link5 = '' #@param {type:"string"}

links = [
   Lora_model_link1,
   Lora_model_link2,
   Lora_model_link3,
   Lora_model_link4,
   Lora_model_link5
]

pth = '/content/lite_colab/models/Lora/'

os.makedirs(pth, exist_ok=True)

for link in links:
    if not link:
        continue  # Skip empty links
    parsed_url = urllib.parse.urlparse(link)
    if not parsed_url.scheme or not parsed_url.netloc:
        print(f'Error: Invalid URL - {link}')
        continue  # Skip invalid URLs
    filename = os.path.join(pth, os.path.basename(parsed_url.path))  # Extract the filename from the URL
    filename_with_extension = f"{filename}.safetensors"  # Append .safetensors extension to the filename
    print(f'Downloading model from {link}...')
    try:
        response = requests.get(link, stream=True)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        # Check if the response headers indicate a valid file
        content_length = response.headers.get('content-length')
        if not content_length or int(content_length) == 0:
            print(f'Error downloading model. File not found or empty: {link}')
            continue  # Skip further processing
        with open(filename_with_extension, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        if os.path.exists(filename_with_extension) and os.path.getsize(filename_with_extension) > 0:
            print('Download complete!')
        else:
            if os.path.exists(filename_with_extension):
                os.remove(filename_with_extension)
            print(f'Error downloading model. File not found or empty: {link}')
    except requests.exceptions.RequestException as e:
        print(f'Error downloading model: {str(e)}')

Download complete!


In [ ]:
#@markdown # **STEP 2.2** ControlNet (optional)
#@markdown ## Run this cell if you need controlnet models.
#@markdown List of used models: https://huggingface.co/lllyasviel/ControlNet-v1-1/tree/main
import os
import requests
from tqdm import tqdm

def download_files(urls, directory):
    for url in urls:
        filename = os.path.basename(url)
        file_path = os.path.join(directory, filename)

        response = requests.get(url, stream=True)
        response.raise_for_status()

        total_size = int(response.headers.get('Content-Length', 0))
        block_size = 8192  # Adjust the block size as per your preference

        progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=filename)

        with open(file_path, 'wb') as file:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                file.write(data)

        progress_bar.close()
        print(f"File downloaded successfully: {file_path}")

# Usage
links = [
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth",
    "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_keypose-fp16.safetensors",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_seg-fp16.safetensors",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_sketch-fp16.safetensors",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_depth-fp16.safetensors",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_canny-fp16.safetensors",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_color-fp16.safetensors",
    "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_style-fp16.safetensors",
    'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_openpose-fp16.safetensors'
]
 # Replace with your list of file URLs

directory = "/content/lite_colab/extensions/lite-kaggle-controlnet/models"  # Replace with your desired directory

download_files(links, directory)



In [ ]:
#@markdown # **STEP 3**
#@markdown ## Run StableDiffusion

!python launch.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --opt-sdp-attention --disable-console-progressbars --no-download-sd-model

In [ ]:
#@markdown # Save "Outputs" to GDrive

from google.colab import drive
drive.mount('/content/gdrive')

!mkdir -p /content/gdrive/MyDrive/lite_colab/outputs
!cp -r /content/lite_colab/outputs /content/gdrive/MyDrive/lite_colab

clear_output()
print('[1;32mDone!')

<form action="https://www.paypal.com/cgi-bin/webscr" method="post" target="_top">
  <input type="hidden" name="cmd" value="_s-xclick" />
  <input type="hidden" name="hosted_button_id" value="MDH8APKUVYW4L" />
  <table>
    <tr>
      <td>
        <input type="hidden" name="on0" value="Thank you for donation!:)"/>
        Thank you for donation!:)
      </td>
    </tr>
    <tr>
      <td>
        <input type="text" name="os0" maxLength="200" />
      </td>
    </tr>
  </table>
  <input type="hidden" name="currency_code" value="USD" />
  <input type="image" src="https://www.paypalobjects.com/en_US/i/btn/btn_paynowCC_LG.gif" border="0" name="submit" title="PayPal - The safer, easier way to pay online!" alt="Buy Now" />
</form>